# Getting started with PyProBE

In [ ]:
%%capture
%pip install matplotlib

In [ ]:
from pprint import pprint

import pyprobe

%matplotlib inline

## Convert data to standard format

Create the cell object and load some data. If this is the first time that the data has been loaded, it must first be converted into the standard format for PyProBE. The `import_from_cycler` method will then add the data directly to the `procedure` dictionary of the cell with the given `procedure_name` as its key.

In [ ]:
# Describe the cell. Required fields are 'Name'.
info_dictionary = {
    "Name": "Sample cell",
    "Chemistry": "NMC622",
    "Nominal Capacity [Ah]": 0.04,
    "Cycler number": 1,
    "Channel number": 1,
}

# Create a cell object
cell = pyprobe.Cell(info=info_dictionary)

data_directory = "../../../tests/sample_data/neware"

cell.import_from_cycler(
    procedure_name="Sample",
    cycler="neware",
    input_data_path=data_directory + "/sample_data_neware.xlsx",
)

If a file named `README.yaml`, sits alongside your data, it will automatically be imported. You can also specify a custom path for this file. The README file contains descriptions of the experiments and steps in the procedure:

In [ ]:
import yaml

with open(data_directory + "/README.yaml") as f:
    pprint(yaml.safe_load(f))

Once loaded, these can be accessed through the `experiment_names` and `step_descriptions` attributes of the procedure:

In [ ]:
print("Experiment names: ", cell.procedure["Sample"].experiment_names)
print("Step Descriptions: ")
pprint(cell.procedure["Sample"].step_descriptions)

Alternatively, if you need to view data quickly and have not prepared a README file, the data will load without one (we will temporarily rename the README file to prevent it being automatically detected):

In [ ]:
import os

os.rename(data_directory + "/README.yaml", data_directory + "/README_bak.yaml")
cell.import_from_cycler(
    procedure_name="Sample Quick",
    cycler="neware",
    input_data_path=data_directory + "/sample_data_neware.xlsx",
)
os.rename(data_directory + "/README_bak.yaml", data_directory + "/README.yaml")

This procedure will have empty `experiment_names` and `step_descriptions` attributes:

In [ ]:
print("Experiment names: ", cell.procedure["Sample Quick"].experiment_names)
print("Step Descriptions: ")
pprint(cell.procedure["Sample Quick"].step_descriptions)

The dashboard can be launched as soon as procedures have been added to the cell (uncomment to run when outside docs environment):

In [ ]:
# pyprobe.launch_dashboard([cell]) # noqa: ERA001

The raw data is accessible as a dataframe with the data property:

In [ ]:
print(cell.procedure["Sample"].data)

Individual columns can be returned as 1D numpy arrays with the `get()` method:

In [ ]:
current = (
    cell.procedure["Sample"].experiment("Break-in Cycles").charge(0).get("Current [A]")
)
print(type(current), current)

Multiple columns can be returned at once:

In [ ]:
current, voltage = (
    cell.procedure["Sample"]
    .experiment("Break-in Cycles")
    .charge(0)
    .get("Current [A]", "Voltage [V]")
)
print("Current = ", current)
print("Voltage = ", voltage)

And different unit can be returned on command:

In [ ]:
current_mA = (
    cell.procedure["Sample"].experiment("Break-in Cycles").charge(0).get("Current [mA]")
)
print("Current [mA] = ", current_mA)

Any part of the procedure can be plotted quickly using the ```plot``` method:

In [ ]:
cell.procedure["Sample"].experiment("Break-in Cycles").plot(
    x="Experiment Time [s]",
    y="Voltage [V]",
)

We can use the `analysis` to further analyse the data. For the `'Break-in Cycles'` we will use the `cycling` analysis module and the functions within. These functions return `Result` objects, so they can be interacted with in the same ways as raw data:

In [ ]:
from pyprobe.analysis import cycling

cycling_summary = cycling.summary(
    input_data=cell.procedure["Sample"].experiment("Break-in Cycles"),
)
print(type(cycling_summary))

print(cycling_summary.data)

And it can be plotted as normal too:

In [ ]:
cycling_summary.plot(x="Capacity Throughput [Ah]", y="Discharge Capacity [mAh]")

As the procedure that we imported without a README file does not contain experiment information, the `Break-in Cycles` will not work on it:

In [ ]:
cycling_summary = cycling.summary(
    input_data=cell.procedure["Sample Quick"].experiment("Break-in Cycles"),
)

However, all other filters will still work as expected.